In [ ]:
import sqlalchemy as sa
import pandas as pd

# SQLite connection string
connection = 'sqlite:///chinook.db' 
engine = sa.create_engine(connection)
con = engine.connect()

albums = pd.read_sql("SELECT * FROM albums", con=con)
sqlite_sequence = pd.read_sql("SELECT * FROM sqlite_sequence", con=con)
artists = pd.read_sql("SELECT * FROM artists", con=con)
customers = pd.read_sql("SELECT * FROM customers", con=con)
employees = pd.read_sql("SELECT * FROM employees", con=con)
genres = pd.read_sql("SELECT * FROM genres", con=con)
invoices = pd.read_sql("SELECT * FROM invoices", con=con)
invoice_items = pd.read_sql("SELECT * FROM invoice_items", con=con)
media_types = pd.read_sql("SELECT * FROM media_types", con=con)
playlists = pd.read_sql("SELECT * FROM playlists", con=con)
playlist_track = pd.read_sql("SELECT * FROM playlist_track", con=con)
tracks = pd.read_sql("SELECT * FROM tracks", con=con)
sqlite_stat1 = pd.read_sql("SELECT * FROM sqlite_stat1", con=con)

con.close()

In [2]:
#Find the total amount spent by each customer on purchases (considering invoices).
total_spent_by_customer = invoices.groupby('CustomerId')['Total'].sum().reset_index()
total_spent_by_customer.columns = ['CustomerId', 'TotalSpent']
print(total_spent_by_customer)

    CustomerId  TotalSpent
0            1       39.62
1            2       37.62
2            3       39.62
3            4       39.62
4            5       40.62
5            6       49.62
6            7       42.62
7            8       37.62
8            9       37.62
9           10       37.62
10          11       37.62
11          12       37.62
12          13       37.62
13          14       37.62
14          15       38.62
15          16       37.62
16          17       39.62
17          18       37.62
18          19       38.62
19          20       39.62
20          21       37.62
21          22       39.62
22          23       37.62
23          24       43.62
24          25       42.62
25          26       47.62
26          27       37.62
27          28       43.62
28          29       37.62
29          30       37.62
30          31       37.62
31          32       37.62
32          33       37.62
33          34       39.62
34          35       37.62
35          36       37.62
3

In [3]:
# Identify the top 5 customers with the highest total purchase amounts.
total_spent_by_customer = invoices.groupby('CustomerId')['Total'].sum().reset_index()
total_spent_by_customer.columns = ['CustomerId', 'TotalSpent']
top_5_customers = total_spent_by_customer.sort_values(by='TotalSpent', ascending=False).head(5)
print(top_5_customers)

    CustomerId  TotalSpent
5            6       49.62
25          26       47.62
56          57       46.62
44          45       45.62
45          46       45.62


In [4]:
# Display the customer ID, name, and the total amount spent for the top 5 customers.
merged = pd.merge(customers, invoices, on='CustomerId', how='inner')
total_spent_by_customer = merged.groupby(['CustomerId', 'FirstName', 'LastName'])['Total'].sum().reset_index()
total_spent_by_customer.columns = ['CustomerId', 'FirstName', 'LastName', 'TotalSpent']
top_5_customers = total_spent_by_customer.sort_values(by='TotalSpent', ascending=False).head(5)
print(top_5_customers)

    CustomerId FirstName    LastName  TotalSpent
5            6    Helena        Holý       49.62
25          26   Richard  Cunningham       47.62
56          57      Luis       Rojas       46.62
44          45  Ladislav      Kovács       45.62
45          46      Hugh    O'Reilly       45.62


In [5]:
# Determine the percentage of customers who prefer to buy individual tracks instead of full albums.
#A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.
#Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).

merged_tracks = pd.merge(invoice_items, tracks, on='TrackId', how='inner')

album_purchases = merged_tracks.groupby(['InvoiceId', 'AlbumId'])['TrackId'].count().reset_index()
album_purchases.columns = ['InvoiceId', 'AlbumId', 'TracksPurchased']

total_tracks_per_album = tracks.groupby('AlbumId')['TrackId'].count().reset_index()
total_tracks_per_album.columns = ['AlbumId', 'TotalTracks']

album_analysis = pd.merge(album_purchases, total_tracks_per_album, on='AlbumId', how='left')

album_analysis['IsFullAlbum'] = album_analysis['TracksPurchased'] == album_analysis['TotalTracks']

full_album_invoices = album_analysis[album_analysis['IsFullAlbum']]['InvoiceId'].unique()

total_invoices = merged_tracks['InvoiceId'].nunique()

individual_track_invoices = total_invoices - len(full_album_invoices)

percentage_individual_tracks = (individual_track_invoices / total_invoices) * 100
percentage_full_albums = 100 - percentage_individual_tracks

print(f"Percentage of customers who prefer individual tracks: {percentage_individual_tracks:.2f}%")
print(f"Percentage of customers who prefer full albums: {percentage_full_albums:.2f}%")

Percentage of customers who prefer individual tracks: 94.42%
Percentage of customers who prefer full albums: 5.58%
